<a href="https://colab.research.google.com/github/JonNData/bw-DS/blob/master/notebooks/many_models_post_here.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [12]:
import spacy
#!python -m spacy download en_core_web_lg
#nlp = spacy.load("en_core_web_lg")
#nlp = spacy.load("en_core_web_md")
#nlp = spacy.load("en_core_web_sm")

import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg
nlp = en_core_web_lg.load()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


## Load data

In [13]:
# load data

!pip install PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"1C4bfP-e2Uoiwc0xZYprS7jar3B9Iy6dB"})  
downloaded.GetContentFile('reddit.tsv')       

In [0]:
data = pd.read_csv('reddit.tsv', sep="\t")
sample = data.sample(frac=0.01, replace=True, random_state=42)
 
print(data.shape)
print(sample.shape)

(1013000, 4)
(10130, 4)


In [0]:
train, test = train_test_split(sample, test_size=0.2, stratify=sample["subreddit"])

train.shape, test.shape

((8104, 4), (2026, 4))

In [0]:
# :: Cleaning :: #

import re
def clean(X):

    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # remove '\\n'
    X['selftext'] = X['selftext'].map(lambda x: re.sub('\\n',' ',str(x)))
    X['subreddit'] = X['subreddit'].map(lambda x: re.sub('\\n',' ',str(x)))
    
    # remove '' 
    X['selftext'] = X['selftext'].map(lambda x: re.sub('<lb>',' ',str(x)))
    X['subreddit'] = X['subreddit'].map(lambda x: re.sub('<lb>',' ',str(x)))
    
    # remove any text starting with User... 
    X['selftext'] = X['selftext'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    X['subreddit'] = X['subreddit'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    
    # remove IP addresses or user IDs
    X['selftext'] =X['selftext'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    X['subreddit'] = X['subreddit'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    
    #remove http links in the text
    X['selftext'] = X['selftext'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))
    X['subreddit'] = X['subreddit'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))
    
    return X

train = clean(train)    
test = clean(test)

In [0]:
train.head()

,id,subreddit,title,selftext
421280,4orqa3,opiates,It's just not fair,"For the second time, I have found Floor Heroin..."
167647,7ewlv6,Costco,Holiday Shipping,Just wanted to ask how shipping is around this...
535426,6w9jvx,bladerunner,Does the Final Cut feel more suspenseful than ...,I recently re-watched the Directors Cut and th...
79973,7o6wsa,amazonecho,Is there any skills that make alexa more talka...,"So yeah, i've been looking after skills that t..."
684647,5vy3fs,cocktails,Honey ginger syrup for penicillins?,I want to make penicillins at a bonfire with s...


In [0]:
X_train = train["selftext"]
X_test = test["selftext"]

y_train = train["subreddit"]
y_test = test["subreddit"]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(8104,) (2026,)
(8104,) (2026,)


In [0]:
X_train.head()

627122    Been using this thing since December '15, look...
917458    i saw a bunch of unfamiliar Costco staff in th...
312727    We have an older toddler and an almost-1-year ...
12100     I've read many lyrical interpretations on TØP,...
825992    The learning curb on these bosses and patterns...
Name: selftext, dtype: object

## Transform data

In [0]:
# spacy tokenizer function to put into tfidf
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip().lower() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [0]:
# Tunning Parameters

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english',
                        sublinear_tf=True,
                        strip_accents='unicode', 
                        ngram_range=(1,2),
                        max_df=.97,
                        min_df=3,
                        max_features=1000,
                        analyzer='word',
                         tokenizer=tokenize)


# Create a vocabulary and get word counts per document
vect = tfidf.fit(X_train)

# these cells are for knn model
# # Get feature names to use as dataframe column headers
# dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# # View Feature Matrix as DataFrame
# dtm.shape
# dtm.head()

In [0]:
# Pickle the vectorizer 
import pickle
with open('vectorizer2.pkl', 'wb') as fin:
  pickle.dump(vect2, fin)

# LSTM model

In [0]:
# Combine for a big vocab
docs1 = X_train + y_train
docs1

128046    Hey guys, I just picked up BL2 a few days ago ...
789852    Hey all,  I'm looking for a Christmas gift for...
78012     About six months ago I tried Ceretropic's Caff...
182220    Hello, I've never posted in this sub but I've ...
850937    I'm looking at on don't understand what to do....
                                ...                        
315929    I need to share my enthousiasm!! Today it is t...
471249    Hi All,  I'm new to the whole crypto scene as ...
432941    Playing skirmish on Crimson in a Pirate's Bane...
426346    Was talking with my Dad the other day about ob...
931775    as salaam alaikum:  I'm married 10yrs. My husb...
Length: 8104, dtype: object

In [0]:
# get the dictionary that will allow encoding and decoding to sequence
from keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(train['selftext'])

print(t.word_index)

{'the': 1, 'i': 2, 'to': 3, 'and': 4, 'a': 5, 'of': 6, 'in': 7, 'it': 8, 'is': 9, 'that': 10, 'my': 11, 'for': 12, 'this': 13, 'but': 14, 'on': 15, 'have': 16, 'with': 17, 'was': 18, 'be': 19, 'you': 20, 'so': 21, 'me': 22, 'or': 23, "i'm": 24, 'if': 25, 'as': 26, 'not': 27, 'just': 28, 'are': 29, 'like': 30, 'at': 31, 'what': 32, 'would': 33, 'from': 34, 'all': 35, 'about': 36, 'do': 37, 'they': 38, 'can': 39, 'some': 40, 'out': 41, 'up': 42, 'any': 43, 'get': 44, 'he': 45, 'an': 46, 'one': 47, 'there': 48, 'we': 49, 'when': 50, 'know': 51, 'how': 52, 'been': 53, 'has': 54, 'am': 55, 'will': 56, 'time': 57, "i've": 58, 'more': 59, 'had': 60, "it's": 61, 'she': 62, 'her': 63, 'now': 64, "don't": 65, 'them': 66, 'really': 67, 'no': 68, '2': 69, 'want': 70, 'also': 71, 'because': 72, 'other': 73, 'by': 74, 'your': 75, 'which': 76, 'only': 77, 'think': 78, '1': 79, 'his': 80, 'then': 81, 'even': 82, 'first': 83, 'could': 84, 'after': 85, 'into': 86, 'new': 87, 'back': 88, 'people': 89, 'm

In [0]:
len(t.word_index.values())

49467

In [0]:
# Use the spacy tokenizer to make X_train into docs
word_list = []
for row in X_train:
  word_list.append(tokenize(row))

word_list

[['second',
  'time',
  'find',
  'floor',
  'heroin',
  'like',
  '2',
  'month',
  'sitting.on',
  'toilet',
  'spot',
  'rock',
  'figure',
  'probably',
  'plaster',
  'crush',
  'snort',
  'drywall',
  'accidentally',
  'couple',
  'time',
  'not.my',
  'proud',
  'moment',
  'counter',
  'press',
  'coin',
  'bam',
  'nice',
  'pile',
  'beige',
  'powder',
  'probably',
  'weigh',
  'quarter',
  'gram',
  'elatef',
  'stuff',
  'like',
  'happen',
  'happy',
  'figure',
  'tolerance',
  'way',
  'easily',
  'stretch',
  '3',
  'shot',
  '',
  'set',
  'guilt',
  'apprehension',
  'motherfucking',
  'consideration',
  'long',
  'term',
  'consequence',
  'motherfucking',
  'action',
  '',
  'high',
  'eternity',
  'deserve',
  'able',
  'night',
  'fun',
  '',
  'deep',
  'know',
  'night',
  'night',
  'becone',
  '',
  'dopefiend',
  'place',
  'throw',
  'toilet',
  'change',
  'mind',
  'stupid',
  'smoke',
  'blunt',
  'beat',
  'feel',
  'like',
  'garbage',
  'mentallly',


In [0]:
# replacing each token in X_train with a sequence #
X_train_seq = []
for list in word_list:
  X_train_seq_intermediate = []
  
  for word in list:
    X_train_seq_intermediate.append(t.word_index[word])
  
  X_train_seq.append(X_train_seq_intermediate)


In [0]:
# transform the target
# Actually don't need to encode the target
labeler = LabelEncoder()
y_train_seq = labeler.fit_transform(y_train)
y_train_seq

In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [0]:
# Set the params of the LSTM model
# Do not change this line. You need the +1 for some reason. 
max_features = len(t.word_index.values()) + 1

maxlen = 75
batch_size = 64

In [0]:
# Make the input an array
X_train_seq_arr = np.array(X_train_seq)
X_train_seq_arr

In [0]:
# Pad the sequence to make uniform entries
x_train1 = sequence.pad_sequences(X_train_seq_arr, maxlen=maxlen)

In [0]:
x_train1

array([[    0,     0,     0, ...,   747,   191,  5066],
       [    0,     0,     0, ...,  1740, 22343,     5],
       [    0,     0,     0, ...,     5,    92,   333],
       ...,
       [ 1165,  1779,  6245, ..., 16376,    98,   855],
       [20449,  6404,   418, ...,    14,   737,   418],
       [   29,  1200,  7253, ...,  2564,   601,  3774]], dtype=int32)

In [0]:
# Build the model
model = Sequential()
# Need this to flatten it to the apt shape
model.add(Embedding(max_features, 128))
# 128 specified by papers/industry. Dropout and recurrent_dropout set our forget params
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# multi class prediction of target y_train
model.add(Dense(len(y_train), activation='softmax'))


model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [0]:
# x_train1 is the padded sequence
model.fit(x_train1, y_train, batch_size=batch_size, epochs=15,
          validation_split=0.2)

#Logistic Regression


In [0]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(n_jobs=10, solver='lbfgs', multi_class='multinomial')

# vectorize features
X_train_vect = vect.transform(X_train)
logreg = logreg.fit(X_train_vect, y_train)

In [0]:
X_test_vect = vect.transform(X_test)
y_pred = logreg.predict(X_test_vect)


In [0]:
from sklearn.metrics import accuracy_score

print('accuracy', accuracy_score(y_pred, y_test))

accuracy 0.09921026653504442


In [0]:
train['selftext']

84082     maybe really become blood brothers in the Germ...
547571    Good morning,  I am an undergrad research stud...
781624    To provide a little bit of background: I've be...
760278    I know this has been asked before, but most of...
538314    Hey everyone! I've started a new story recentl...
                                ...                        
773815    Apart from the obvious - why does everyone cal...
39499     So I haven't seen a doctor yet (though I need ...
370191    I just got the Wacom Intuos Draw and I have tr...
519777    Would be fun to know your preferred Arc/Episod...
879381    I don't intend to turn this into a thread bash...
Name: selftext, Length: 8104, dtype: object

In [0]:
# Test it on sample text
sample_text = """
Hello all, throw away for obvious reasons.

So, I'm a big pet lover. We have several dogs, reptiles, frogs and small animals.

I work in the pet care industry and have recently lost my job due to the shutdown.

That being the case, I've been home a lot more often, and I noticed the animals aren't attached the him like me. I thought nothing of it, as I care for them mainly and had most of them before we got together.

Our pets have their own room, and yesterday I went to let out dogs out and our 5lb Chihuahua mix was limping and putting no weight on his leg. My husband jokingly said a few minutes before I checked on him, "I got Rufus to stop scratching the door."

I thought nothing of it until my poor dog was unable to move.

I took him to the vet and they did x-rays and let me know Rufus's hip was broken. They said being so small it could be because he jumped off the furniture and hurt himself, as he's rather old as well.

My husband later admitted that he lost his temper, picked Rufus up the the scruff, and beat him.

He feels incredibly guilty and wants nothing to do with any of the pets now.

I found out yesterday I'm pregnant, and I'm worried he may loose his temper with a newborn much worse then Rufus's constant scratching.

What do I do? I love this man and we've been married half a decade now.
              """

In [0]:
# sample text must be iterable to vectorize
logreg.predict(vect.transform([sample_text])

array(['gaymers'], dtype=object)

# Stochastic Gradient Descent Classifier

In [0]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
# use the previously vectorized features
sgd = sgd.fit(X_train_vect, y_train)
y_pred = sgd.predict(X_test_vect)

print('SGDClassifier accuracy', accuracy_score(y_pred, y_test))

SGDClassifier accuracy 0.13968410661401776


In [0]:
sgd.predict(vect.transform([sample_text]))

array(['Petscop'], dtype='<U21')

In [0]:
# Pickle the SGD Classifier
with open('SGDClassifier.pkl', 'wb') as fin:
  pickle.dump(sgd, fin)

# New hypothesis
#### The reason all of our  models are doing poorly is because there's too many subreddits with small number of instances. Let's do it all again.

In [0]:
sample = data.sample(frac=0.015, replace=True, random_state=42)
train, test = train_test_split(sample, test_size=0.15, stratify=sample["subreddit"])

train.shape, test.shape

((12915, 4), (2280, 4))

In [0]:
train = clean(train)    
test = clean(test)
X_train = train["selftext"]
X_test = test["selftext"]

y_train = train["subreddit"]
y_test = test["subreddit"]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(12915,) (2280,)
(12915,) (2280,)


In [0]:
# pd.set_option('display.max_rows', 1000)
# y_train.value_counts().head(1000)

# Ok the problem is actually there are too many subreddits just in general, 
# and we don't even have the main ones in here. Like r/aww, r/investing

bigboobproblems          64
sailing                  62
MoviePassClub            60
DimensionalJumping       59
SleepApnea               59
secretsanta              59
osx                      58
Corsair                  58
baldursgate              58
Conservative             57
stopsmoking              57
korea                    57
blackmirror              57
HouseOfCards             57
watercooling             57
walkingwarrobots         56
Narcolepsy               56
hiphopheads              55
photography              55
gorillaz                 54
amiibo                   54
malaysia                 54
HomeDepot                54
Volkswagen               54
bettafish                54
danganronpa              54
arrow                    54
RedditLaqueristas        54
peacecorps               54
personalfinance          54
Rowing                   54
dbz                      54
Cubers                   54
workflow                 54
baduk                    54
audible             

In [0]:
tfidf1 = TfidfVectorizer(stop_words='english',
                        sublinear_tf=True,
                        strip_accents='unicode', 
                        ngram_range=(1,2),
                        max_df=.97,
                        min_df=3,
                        max_features=1000,
                        analyzer='word',
                        token_pattern=r'\w{2,}',
                        tokenizer=tokenize)

vect = tfidf1.fit(X_train)

In [0]:
# Vectorize the Xs
X_train_vect = vect.transform(X_train)
X_test_vect = vect.transform(X_test)


In [0]:
# Logistic Regression

logreg = LogisticRegression(n_jobs=10, solver='lbfgs', multi_class='multinomial')

logreg = logreg.fit(X_train_vect, y_train)

y_pred = logreg.predict(X_test_vect)
print('Logistic accuracy:', accuracy_score(y_pred, y_test))

In [0]:
# Stochastic Gradient Descent
sgd = SGDClassifier()
sgd = sgd.fit(X_train_vect, y_train)
y_pred = sgd.predict(X_test_vect)

print('SGDClassifier accuracy', accuracy_score(y_pred, y_test))

# Try it with pickles


In [0]:
# load the pickle models
import pickle

with open('vectorizer2.pkl', 'rb') as file:
    vec_pickle = pickle.load(file)

with open('SGDClassifier_pickle.pkl', 'rb') as file1:
    clf_pickle = pickle.load(file1)

In [6]:
# Test with sample input

sample_text = """
If you have Final Fantasy 7 Remake preordered and really want it, go pick it up ASAP. Our district was told we have to send them out for online orders otherwise we would be closed again for not fulfilling these orders. They don't care, they just want the sales. Can't wait to deal with the aftermath.
              """
text_vec = vec_pickle.transform([sample_text])
output = clf_pickle.predict(text_vec)
    
type(output)

numpy.ndarray

In [7]:
output[0]

'Ghosts'

In [0]:
# Notes: it will come in JSON so use jsonify

## Model

Knn


In [0]:
# algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}
nn = NearestNeighbors(n_neighbors=10, algorithm='brute', n_jobs=-1)

# Fit the model on TFidf Vectors
nn.fit(dtm)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=10, p=2,
                 radius=1.0)

In [0]:
nn.kneighbors([dtm.iloc[0].values])

(array([[0.        , 1.13874581, 1.20226557, 1.21065594, 1.23113375,
         1.2352659 , 1.24365648, 1.24431962, 1.24488678, 1.24901069]]),
 array([[   0,  294, 3480, 6215,  390, 3725, 2117, 2825, 5672, 3819]]))

In [0]:
# Query Using kneighbors 
nn.kneighbors([dtm.iloc[42]])

(array([[2.10734243e-08, 1.04315344e+00, 1.04397239e+00, 1.07311766e+00,
         1.07580780e+00, 1.07972784e+00, 1.08015754e+00, 1.09048932e+00,
         1.09634997e+00, 1.09756603e+00]]),
 array([[  42, 1835, 3886, 5621, 1095, 3094, 7645, 3426, 7345, 5920]]))

## Test

In [0]:
# https://www.reddit.com/r/learnprogramming/comments/g99at4/i_printed_hello_world_in_cobol/

test_input = """
I’m not much of a programmer, but when I saw that the world needs COBOL programmers right now, 
I thought I would do my best to help out, even though I knew nothing about the language. I’ve 
spent way too many hours over the past two weeks trying to get my system configured just to 
compile and run COBOL code. It might not seem like a big deal, but seeing those two words on 
the system output makes me feel like I can do anything!
"""

test_sparse = tfidf.transform([test_input])

In [0]:
test_array = nn.kneighbors(test_sparse.todense(), n_neighbors=10)
test_array

(array([[1.18033871, 1.18590088, 1.20771511, 1.20865218, 1.20988773,
         1.21075963, 1.21339007, 1.21622075, 1.22106848, 1.22322952]]),
 array([[4575, 2399, 1868, 4506, 2257,  465, 2391, 3063, 2330,  954]]))

In [0]:
# Extract the second item in the outer array
# This is the list of the review indices that are 'closest' to input

rec_id_list = test_array[1][0]
rec_id_list

array([4575, 2399, 1868, 4506, 2257,  465, 2391, 3063, 2330,  954])

In [0]:
data.iloc[rec_id_list]["subreddit"]

4575    twentyonepilots
2399               GMAT
1868             weezer
4506              wacom
2257            osugame
465              xxketo
2391       transformers
3063            Stellar
2330               UFOs
954         foxholegame
Name: subreddit, dtype: object

In [0]:
input_two = """

Michael Jordan on Isiah Thomas: "Whatever he says now, you know it wasn't his true actions then. 
He's had time to think about it. Or, the reaction of the public, that's kind of changed his 
perspective of it. You can show me anything you want. There's no way you can convince me he wasn't an asshole.
"""

In [0]:
def recommend(req, n=10):
    """Function to recommend top n subreddits given a request."""
    # Create vector from request
    req_vec = tfidf.transform([req])

    # Get indexes for n nearest neighbors
    top_id = nn.kneighbors(req_vec.todense(), n_neighbors=n)[1][0]

    # Index-locate the neighbors in original dataframe
    top_array = data.iloc[top_id]["subreddit"]

    return top_array

In [0]:
result = recommend(input_two)
result

5579                 cigars
7646    lawschooladmissions
7344                    wls
4025               funkopop
6748           LongDistance
4890                parrots
4948         suggestmeabook
2550               Warframe
3923               ArcherFX
311                  Tinder
Name: subreddit, dtype: object

In [0]:
# Results are...lol

# Use current top trending subreddits + post and retrain entrire model = PRAW api. 